In [1]:
import pandas as pd

In [2]:
bores = pd.read_csv('NGIS_Bore.csv', usecols=[
    'HydroID',
    'HydroCode',
    'StateTerritory',
    'BoreDepth',
    'DrilledDate',
    'Latitude',
    'Longitude',
    'LandElev',
    'FTypeClass' # commerical, industrial, domestic etc..
])

# JOIN ON HYDRO ID ONLY???


/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
bores = bores.rename(columns={
    'HydroID': 'hydro_id',
    'HydroCode': 'hydro_code',
    'StateTerritory': 'state',
    'BoreDepth': 'bore_depth',
    'DrilledDate': 'drilled_date',
    'Latitude': 'lat',
    'Longitude': 'long',
    'LandElev': 'land_elevation',
    'FTypeClass': 'type_of_use'
})

#bores.set_index(['bore_id', 'state', 'hydro_id'], inplace=True, verify_integrity=True)

In [4]:
bores.set_index('hydro_id', inplace=True, verify_integrity=True)

In [8]:
for state in ['NSW', 'QLD', 'NT', 'SA', 'TAS', 'VIC', 'WA']:
    print(f'Working on {state}')
    ## SALINITY
    salinity = pd.read_csv(f'{state}/salinity_{state}.csv', usecols=[
        'hydroid', 'bore_id', 'bore_date', 'uom', 'result'
    ])
    salinity = salinity.rename(columns={
        'hydroid': 'hydro_id',
        'bore_date': 'last_salinity_measurement'
    })
    # only take latest salinity measurements
    salinity = salinity.sort_values('last_salinity_measurement', ascending=False).drop_duplicates('hydro_id')
    salinity.set_index('hydro_id', inplace=True, verify_integrity=True)
    
    df = salinity.merge(bores, on='hydro_id', how='left')
    
    ## LEVELS
    levels = pd.read_csv(f'{state}/level_{state}.csv', usecols=[
        'hydroid', 'bore_id', 'bore_date', 'obs_point_datum', 'result'
    ]) 
    
    levels = levels.rename(columns={
        'hydroid': 'hydro_id',
        'bore_date': 'last_level_measurement'
    })
    
    levels = levels.sort_values('last_level_measurement', ascending=False).drop_duplicates('hydro_id')
    df2 = df.merge(levels, on='bore_id', how='left')
    df2.to_csv(f'output_{state}.csv', index=False)
    df2 = df2.fillna('')
    df_to_geojson(state, df2)

    
    

Working on NSW
Working on QLD
Working on NT
Working on SA
Working on TAS
Working on VIC
Working on WA


In [6]:
import geojson

In [7]:
def df_to_geojson(name, df):
    old_df = df
    features = []
    simple_features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["long"],
                                                    X["lat"])),
                            properties=dict(id=X["bore_id"], 
                                            icon="drinking-water-15", 
                                            last_salinity_measurement=X["last_salinity_measurement"],
                                            salinity_uom=X["uom"],
                                            salinity=X["result_x"],
                                            bore_depth=X["bore_depth"],
                                            drilled_date=X["drilled_date"],
                                            land_elevation=X["land_elevation"],
                                            type_of_use=X["type_of_use"],
                                            last_level_measurement=X["last_level_measurement"],
                                            obs_point_datum=X["obs_point_datum"],
                                            level=X["result_y"]
                                           )))
    insert_features_simple = lambda X: simple_features.append(
        geojson.Feature(geometry=geojson.Point((X["long"],
                                                X["lat"])),
                        properties=dict(id=X["bore_id"], icon="drinking-water-15")
                       )
    )

    df.apply(insert_features, axis=1)
    old_df.apply(insert_features_simple, axis=1)
    

    with open(f'{name}.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)
        
    with open(f'{name}_simple.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(simple_features), fp, sort_keys=True, ensure_ascii=False)